<span style=" font-size:1.3em;">**Decision Trees for Classification**</span>

In [3]:
from scipy.io import loadmat
import csv
import pandas as pd

spam_data = loadmat("spam_data.mat")
spam_train = spam_data['training_data']
spam_labels = spam_data['training_labels'][0]
spam_test = spam_data['test_data']


<span style=" font-size:1.3em;">**Implement Decision Trees**</span>


In [4]:

"""
This is the starter code and some suggested architecture we provide you with. 
But feel free to do any modifications as you wish or just completely ignore 
all of them and have your own implementations.
"""
from collections import Counter
import numpy as np
from numpy import genfromtxt
import scipy.io
from scipy import stats
import random

class DecisionTree:

    def __init__(self, data, y, indices, split_rule, label = None):
        """
        TODO: initialization of a decision tree
        """
        self.data = data
        self.y = y
        self.indices = indices
        self.split_rule = split_rule
        self.label = None
        self.left = None
        self.right = None
        
    @staticmethod
    def entropy(y):
        """
        TODO: implement a method that calculates the entropy given all the labels
        """
        proportions = np.unique(y, return_counts = True)[1]/len(y)
        proportions = proportions[proportions!=0]

        return -sum(proportions*np.log2(proportions))
    @staticmethod
    def binary_entropy(num_c, total):
        if total == 0:
            return total
        p = num_c/total
        left = -p*np.log2(p) if p != 0 else 0
        right = -(1-p)*np.log2(1-p) if (1-p) != 0 else 0
        return left + right
    

    @staticmethod
    def H_after(left_c, left_length, right_c, right_length):
        n = left_length + right_length
        
        HSl = DecisionTree.binary_entropy(left_c, left_length)
        HSr = DecisionTree.binary_entropy(right_c, right_length)
        return (left_length*HSl + right_length*HSr)/n
    @staticmethod
    def map_sort(X, y):
        X, y = list(X), list(y)
        zipped = np.array(sorted(zip(X, y)))
        return zipped[:,0], zipped[:,1]
    
    @staticmethod
    def segmenter(X, y, random_forest = False):
        """
        TODO: compute entropy gain for all single-dimension splits,
        return the feature and the threshold for the split that
        has maximum gain
        """
        n = len(y)
        max_gain = -1
        best_feature = 0
        best_threshold = 0
        f = len(X[0])
        H = DecisionTree.entropy(y)
        
        if random_forest:
            features = random.sample(range(f), int(np.sqrt(f)))
            X = np.array(random.choices(X, k = len(X)))
        else:
            features = range(f)
            
        for feat in features:
            entropies = []
            column = X[:, feat]
            
            left_length, right_length = 0, len(column)
            left_c, right_c = 0, sum(y == y[0])
            
            sorted_X, sorted_y = DecisionTree.map_sort(column, y)

            c = y[0]
            for i in range(n):
                left_length += 1
                right_length -= 1
                
                if sorted_y[i] == c:
                    left_c += 1
                    right_c -= 1
                    
                if (i < n - 1) and (sorted_X[i] == sorted_X[i+1]):
                    entropies.append(100)
                else:
                    entropies.append(DecisionTree.H_after(left_c, left_length, right_c, right_length))
                    
            j = np.argmin(entropies)
            info_gain = H - entropies[j]

            if info_gain >= max_gain:
                best_feature = feat
                best_threshold = sorted_X[j]
                max_gain = info_gain

                    
        return (best_feature, best_threshold)
                
    def split(self, feature, thresh):
        """
        TODO: implement a method that return a split of the dataset given an index of the feature and
        a threshold for it
        """
        if self.label:
            return
        
        indices = self.indices
        left, right = [], []
        c = self.y[0]
        left_c = 0
        right_c = 0
        
        for i in indices:
            if self.data[i, feature] > thresh:
                left.append(i)
                if self.data[i, feature] == c:
                    left_c+=1
            else:
                right.append(i)
                if self.data[i, feature] == c:
                    right_c+=1

        labels_left, labels_right = self.y[left], self.y[right]
        label_l, label_r = None, None
        
        H = DecisionTree.entropy(self.y)
        H_after = DecisionTree.H_after(left_c, len(labels_left), right_c, len(labels_right))
        info_gain = H - H_after
        
        if info_gain <= 1e-20 and len(labels_left) > 0:
            label_l = round(np.mean(labels_left))
        if info_gain <= 1e-20 and len(labels_right) > 0:
            label_r = round(np.mean(labels_right))

        len_left, len_right = len(left), len(right)
        
        if len_left != 0:
            self.left = DecisionTree(self.data, self.y, left, None, label_l)
        if len_right != 0:
            self.right = DecisionTree(self.data, self.y, right, None, label_r)
        return
    
    def fit(self, depth = 0, d = False, random_forest = False):
        """
        TODO: fit the model to a training set. Think about what would be 
        your stopping criteria
        """
        if (self.label != None) or (self == None):
            return
        
        X = self.data[self.indices]
        y = self.y[self.indices]
        
        if d:
            if (depth == 0):
                if len(np.unique(y)) == 1:
                    self.label = self.y[0]
                else:
                    self.label = round(np.mean(y))
                return
            else:
                feature, thresh = DecisionTree.segmenter(X, y, random_forest)
                self.split_rule = (feature, thresh)
                self.split(feature, thresh)
                if self.left:
                    self.left.fit(depth - 1, True)
                if self.right:
                    self.right.fit(depth - 1, True)
        else:
            feature, thresh = DecisionTree.segmenter(X, y, random_forest)
            self.split(feature, thresh)
            
            self.left.fit()
            self.right.fit()


    def predict(self, X):
        """
        TODO: predict the labels for input data 
        """
                
        predictions = []
        for x in X:
            ##go through the tree
            temp = self
            
            while temp:
                if temp.label != None:
                    predictions.append(temp.label)
                    break
                        
                feature, threshold = temp.split_rule
                if x[feature] > threshold:
                    if temp.left == None:
                        predictions.append(round(np.mean(temp.y[temp.indices])))
                        break
                    else:
                        temp = temp.left
                else:
                    if temp.right == None:
                        predictions.append(round(np.mean(temp.y[temp.indices])))
                        break
                    else:
                        temp = temp.right
                        
        return predictions

    def __repr__(self):
        """
        TODO: one way to visualize the decision tree is to write out a __repr__ method
        that returns the string representation of a tree. Think about how to visualize 
        a tree structure. You might have seen this before in CS61A.
        """
        string = "(" + str(self.split_rule) + ")\n"
        
        if self.left:
            string += "left:" + repr(self.left)
        else:
            string += "left No Node\n"
        if self.right:
            string += "right:" + repr(self.right)
        else:
            string += "right No Node\n"
        return string
            
    

In [5]:
def l2_normalize(lst):
    return lst.astype(np.double)/np.linalg.norm(lst)

<span style=" font-size:1.3em;">**Random Forests**</span>

In [6]:
def random_forest_predict(predict_data, num_trees, depth, train_data, labels):
    predicted = np.array([0.0]*len(predict_data)).astype('float')
    for i in range(num_trees):
        tree = DecisionTree(train_data, labels, range(len(labels)), None)
        tree.fit(depth, True, True)
        predicted += np.array(tree.predict(predict_data)).astype('float')
    return np.round(predicted/num_trees)

<span style=" font-size:1.3em;">**Implementation Details**</span>

1. Categorical features were converted to numbers and missing values  
were replaced with most frequently appearing values using sklearn's  
Implementer
2. Stopping criterion was the depth. Leaves are created when  
information gain is less than or equal to 1-e20  
3. Random Forests were implemented with the decision tree,  
but randomly selected root(M) features and then the forest with  
n number of trees' predictions are averaged out to give majority prediction.  
4. I sped up the training using the trick taught in lecture using the  
sorted labels to determine thresholds that minimizes entropy to maximize  
info gain.  
5. I normalized vectors and that improved accuracy.

<span style=" font-size:1.3em;">**Performance Evaluation**</span>

In [7]:
##Split
def train_valid_split(data, labels, seed = 12, percent_train = 0.8):
    n = len(data)
    train_count = int(percent_train*n)
    random.seed(seed)
    random_indices = random.sample(range(n), len(labels))
    train_indices = random_indices[:train_count]
    validation_indices = random_indices[train_count:]

    training_data = data[train_indices]
    training_labels = labels[train_indices]

    validation_data = data[validation_indices]
    validation_labels = labels[validation_indices]
    
    return training_data, training_labels, validation_data, validation_labels

training_data_spam, training_labels_spam, validation_data_spam, validation_labels_spam = train_valid_split(spam_train, spam_labels)
training_data_spam = l2_normalize(training_data_spam)
validation_data_spam = l2_normalize(validation_data_spam)
spam_test = l2_normalize(spam_test)

In [9]:
from sklearn.impute import SimpleImputer
#Data cleaning for Titanic

titanic_training = pd.read_csv("titanic_training.csv")
titanic_test = pd.read_csv("titanic_testing_data.csv")


titanic_training['sex'] = [1 if val=="female" else 0 for val in titanic_training['sex']]
titanic_test['sex'] = [1 if val=="female" else 0 for val in titanic_test['sex']]

titanic_training_labels = titanic_training['survived'].copy()
titanic_training_labels[np.isnan(titanic_training_labels)] = 1
titanic_training = titanic_training.drop(columns=["ticket", "cabin", "survived"])
titanic_test = titanic_test.drop(columns = ['cabin', 'ticket'])

replace = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
replace.fit(titanic_training)

titanic_training = pd.DataFrame(replace.transform(titanic_training), index = titanic_training.index, columns = titanic_training.columns)
titanic_test = pd.DataFrame(replace.transform(titanic_test), index = titanic_test.index, columns = titanic_test.columns)

titanic_training = titanic_training.join(pd.get_dummies(titanic_training['embarked'])).drop(columns = ['embarked'])
titanic_test = titanic_test.join(pd.get_dummies(titanic_test['embarked'])).drop(columns = ['embarked'])
titanic_training = titanic_training.values.astype(float)
titanic_labels = titanic_training_labels.values.astype(float)
titanic_test = titanic_test.values.astype(float)

training_data_titanic, training_labels_titanic, validation_data_titanic, validation_labels_titanic = train_valid_split(titanic_training, titanic_labels)


In [10]:
##Decision Tree Accuracies (Takes a while to run)
decision_tree = DecisionTree(training_data_spam, training_labels_spam, range(len(training_labels_spam)), None)
decision_tree.fit(3, True)
training_spam = np.mean(decision_tree.predict(training_data_spam) == training_labels_spam)
validation_spam = np.mean(decision_tree.predict(validation_data_spam) == validation_labels_spam)

forest_training_spam = random_forest_predict(training_data_spam, 20, 4, training_data_spam, training_labels_spam)
forest_validation_spam = random_forest_predict(validation_data_spam, 20, 4, training_data_spam, training_labels_spam)
training_spam_forest = np.mean(forest_training_spam == training_labels_spam)
validation_spam_forest = np.mean(forest_validation_spam == validation_labels_spam)

decision_tree = DecisionTree(training_data_titanic, training_labels_titanic, range(len(training_labels_titanic)), None)
decision_tree.fit(3, True)
training_titanic = np.mean(decision_tree.predict(training_data_titanic) == training_labels_titanic)
validation_titanic = np.mean(decision_tree.predict(validation_data_titanic) == validation_labels_titanic)

forest_training_titanic = random_forest_predict(training_data_titanic, 20, 4, training_data_titanic, training_labels_titanic)
forest_validation_titanic = random_forest_predict(validation_data_titanic, 20, 4, training_data_titanic, training_labels_titanic)
training_titanic_forest = np.mean(forest_training_titanic == training_labels_titanic)
validation_titanic_forest = np.mean(forest_validation_titanic == validation_labels_titanic)

print('training for spam Decision Tree: ', training_spam)
print('validation for spam Decision Tree: ', validation_spam)
print('training for spam Forest: ', training_spam_forest)
print('validation for spam Forest: ', validation_spam_forest)

print('training for titanic Decision Tree: ', training_titanic)
print('validation for titanic Decision Tree: ', validation_titanic)
print('training for titanic Forest: ', training_titanic_forest)
print('validation for titanic Forest: ', validation_titanic_forest)

training for spam Decision Tree:  0.7918781725888325
validation for spam Decision Tree:  0.8067632850241546
training for spam Forest:  0.7918781725888325
validation for spam Forest:  0.8067632850241546
training for titanic Decision Tree:  0.77125
validation for titanic Decision Tree:  0.705
training for titanic Forest:  0.77
validation for titanic Forest:  0.7


In [ ]:
###Takes a LONG TIME to run
tree_training_errors = []
tree_validation_errors = []

depths = range(1,41)

for depth in depths:
    test_tree = DecisionTree(training_data_spam, training_labels_spam, range(len(training_labels_spam)), None)
    test_tree.fit(depth, True)

    tree_validation_errors.append(np.mean(test_tree.predict(validation_data_spam) == validation_labels_spam))
    tree_training_errors.append(np.mean(test_tree.predict(training_data_spam) == training_labels_spam))
    